# Approach 1 without correlation check

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [2]:
Data=pd.read_csv("data.csv")
Data.head()

FileNotFoundError: ignored

In [ ]:
Data=Data.drop(columns=["id"])
Data.isnull().sum()


In [ ]:
Data=Data.drop(columns=["Unnamed: 32"])
Data.isnull().sum()
Data.head()

In [ ]:
len(Data.dtypes)

In [ ]:
columns=list(Data.columns)
count=1
#plt.figure(figsize=(22, 20), dpi=80)
#for i in range(6):
#    for j in range(5):
#        uniques=Data[columns[count]]
#        plt.subplot(6, 5, count) 
#        plt.hist(uniques,bins=40)
#        plt.title(columns[count])
#        count=count+1
#plt.tight_layout(pad=1.0)

In [ ]:
#plt.figure(figsize=(22, 20), dpi=80)
#fig, axes = plt.subplots(8,4,figsize=(25,25))
#for i in range(len(columns)-1):
#    Data.boxplot(column=[columns[i+1]],ax=axes.flatten()[i])

In [ ]:
print(len(Data))
for i in range(len(columns)-1):
    m=Data[columns[i+1]].mean()
    std=Data[columns[i+1]].std()
    Data=Data[(Data[columns[i+1]]>m-3*std) & (Data[columns[i+1]]<m+3*std)]

In [ ]:
train=Data.sample(frac=0.8,random_state=200)
test=Data.drop(train.index)
Xtrain=np.array(train[columns[1:len(columns)]])
Ytrain=np.array(pd.get_dummies(train[columns[0]]))
Xtest=np.array(test[columns[1:len(columns)]])
Ytest=np.array(pd.get_dummies(test[columns[0]]))

model = tf.keras.Sequential([
  tf.keras.layers.Dense(64,activation="sigmoid"),
  tf.keras.layers.Dense(64,activation="sigmoid"),
  tf.keras.layers.Dense(2)
])

model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())

model.fit(Xtrain, Ytrain, epochs=12,verbose=0)
A=model.predict(Xtest)

(1-abs(A.argmax(axis=1)-Ytest[:,1]).sum()/len(A))*100


# Approach1:Eval

In [ ]:
(1-abs(A.argmax(axis=1)-Ytest[:,1]).sum()/len(A))*100

90.05847953216374

# Approach2: With Correlation Check

In [ ]:
Data=pd.read_csv("data.csv")
#Data.head()
Data=Data.drop(columns=["id"])
Data=Data.drop(columns=["Unnamed: 32"])
#Data.isnull().sum()
#Data.head()

In [ ]:
Data=pd.get_dummies(Data)
Data=Data.drop(columns=["diagnosis_B"])
#Data.head()

In [ ]:
cor=Data.corr()
#print(cor.diagnosis_M)
Index=cor.diagnosis_M[(cor.diagnosis_M>0.78) | (cor.diagnosis_M<-0.78)].index
Data=Data[Index]
Data.corr()

,perimeter_worst,concave points_worst,diagnosis_M
perimeter_worst,1.000000,0.816322,0.782914
concave points_worst,0.816322,1.000000,0.793566
diagnosis_M,0.782914,0.793566,1.000000


In [ ]:
Data=Data[Index]
Data.head()

,perimeter_worst,concave points_worst,diagnosis_M
0,184.60,0.2654,1
1,158.80,0.1860,1
2,152.50,0.2430,1
3,98.87,0.2575,1
4,152.20,0.1625,1


In [ ]:
#print(len(Data))
#for i in range(len(columns)-1):
#    m=Data[columns[i+1]].mean()
#    std=Data[columns[i+1]].std()
#    Data=Data[(Data[columns[i+1]]>m-3*std) & (Data[columns[i+1]]<m+3*std)]

In [ ]:
columns=list(Data.columns)
Data=Data.sample(frac=1)
train=Data.sample(frac=0.7)
test=Data.drop(train.index)
Xtrain=np.array(train[columns[0:len(columns)-1]])
Ytrain=np.array(pd.get_dummies(train[columns[-1]]))
Xtest=np.array(test[columns[0:len(columns)-1]])
Ytest=np.array(pd.get_dummies(test[columns[-1]]))

model = tf.keras.Sequential([
  tf.keras.layers.Dense(80,activation="sigmoid"),
  tf.keras.layers.Dense(80,activation="sigmoid"),
  tf.keras.layers.Dense(2)
])

model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())

model.fit(Xtrain, Ytrain, epochs=12,verbose=0)
A=model.predict(Xtest)

(1-abs(A.argmax(axis=1)-Ytest[:,1]).sum()/len(A))*100

92.98245614035088

# Approach2:Eval

In [ ]:
(1-abs(A.argmax(axis=1)-Ytest[:,1]).sum()/len(A))*100

92.98245614035088

# Approach3

In [ ]:
!pip install tensorflow_decision_forests
import tensorflow_decision_forests as tfdf
import pandas
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 17.1 MB 254 kB/s 


In [ ]:
Data=pd.read_csv("data.csv")
#Data.head()
Data=Data.drop(columns=["id"])
Data=Data.drop(columns=["Unnamed: 32"])
#Data.isnull().sum()
#Data.head()

In [ ]:
#Split the Data in Train dataset and Test datast
Data=Data.sample(frac=1)
train_df = Data.sample(frac=0.75,random_state=200)
test_df=Data.drop(train_df.index)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="diagnosis")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="diagnosis")
model = tfdf.keras.GradientBoostedTreesModel(num_trees=500, max_depth=8)
model.fit(train_ds)
model.compile(metrics=["accuracy"])
print("Accuracy of the Model:",model.evaluate(test_ds)[1])

3/3 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.9718
Accuracy of the Model: 0.9718309640884399


# Approach3:Eval

In [ ]:
print("Accuracy of the Model:",model.evaluate(test_ds)[1])

3/3 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.9718
Accuracy of the Model: 0.9718309640884399
